In [5]:
import pandas as pd
import math
from mekerere.IOUtility import crop_and_resize_on_bounding_box
import cv2 as cv
import shutil
import os

# Crop and resize images based on bounding box

In [6]:
ROOT_DATA_PATH = "Data"
IMAGE_DIR = f"{ROOT_DATA_PATH}/Train_Images"
OUTPUT_CROPPED_DIR = f"{ROOT_DATA_PATH}/Cropped_Images"
training_data_file = f"{ROOT_DATA_PATH}/Train.csv"
cropped_data_file = f"{ROOT_DATA_PATH}/Cropped.csv"

In [7]:
def crop_resize_save_training_images():
    training_df = pd.read_csv(training_data_file)
    image_filenames = os.listdir(IMAGE_DIR)
    cropped_training_file_data = []
    for image_filename in image_filenames:
        dataframes = training_df.loc[image_filename.replace('.jpg','') == training_df['Image_ID']]
        index = 0
        for _, dataframe in dataframes.iterrows():
            image = crop_and_resize_on_bounding_box(
                filename = f"{IMAGE_DIR}/{image_filename}",
                x_coord = math.ceil(dataframe['xmin']),
                y_coord = math.ceil(dataframe['ymin']),
                width = math.ceil(dataframe['width']),
                height = math.ceil(dataframe['height'])
            )
            output_image_name = dataframe['Image_ID']
            if len(dataframes) > 1:
                index+=1
                output_image_name += f"_{'{:0>2}'.format(index)}"

            cv.imwrite(f"{OUTPUT_CROPPED_DIR}/{output_image_name}.jpg", image)
            dataframe["Image_ID"] = output_image_name
            cropped_training_file_data.append((dataframe["Image_ID"], dataframe["class"]))

    cropped_data = pd.DataFrame(cropped_training_file_data, columns=['Image_ID', 'class'])
    cropped_data.to_csv(cropped_data_file)

# Split image data into different datasets

In [8]:
IMAGE_SIZE = 500
TRAINING_SEED = 1
TRAINING_PROPORTION = 0.7
TESTING_PROPORTION = 0.2

TRAINING_IMAGE_DIR = F"{ROOT_DATA_PATH}/Train"
TEST_IMAGE_DIR = F"{ROOT_DATA_PATH}/Test"
VALIDATION_IMAGE_DIR = F"{ROOT_DATA_PATH}/Validation"

In [9]:
def split_dataset():
    traning_filenames = os.listdir(IMAGE_DIR)
    random.seed(TRAINING_SEED)
    total_images = len(traning_filenames)

    for index, training_filename in enumerate(traning_filenames):
        percentage = index / total_images
        if percentage < TRAINING_PROPORTION:
            destination = TRAINING_IMAGE_DIR
        elif percentage < TRAINING_PROPORTION + TESTING_PROPORTION:
            destination = TEST_IMAGE_DIR
        else:
            destination = VALIDATION_IMAGE_DIR
        
        shutil.move(f"{IMAGE_DIR}/{training_filename}", f"{destination}/{training_filename}")